In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForTokenClassification, AdamW
from transformers import get_scheduler
import torch
from torch.utils.data import DataLoader, Dataset
from seqeval.metrics import classification_report
import sqlite3


In [4]:
from google.colab import files

# Upload file
uploaded = files.upload()



Saving channels_to_crawls.csv to channels_to_crawls.csv


In [5]:
import pandas as pd

# Load the uploaded file
channels_df = pd.read_csv('channels_to_crawls.csv', header=None)

# Use the first column as the channel list
channel_list = channels_df[0].tolist()
print(channel_list)


['@ZemenExpress', '@nevacomputer', '@meneshayeofficial', '@ethio_brand_collection', '@Leyueqa', '@sinayelj', '@Shewabrand', '@helloomarketethiopia', '@modernshoppingcenter', '@qnashcom', '@Fashiontera', '@kuruwear', '@gebeyaadama', '@MerttEka', '@forfreemarket', '@classybrands', '@marakibrand', '@aradabrand2', '@marakisat2', '@belaclassic', '@AwasMart', '@qnashcom', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n

In [6]:
import nest_asyncio
nest_asyncio.apply()  # Allow nested event loops


In [7]:
import time
import asyncio

async def fetch_with_retry(channel_info, limit=100, retries=3):
    attempt = 0
    while attempt < retries:
        try:
            return await fetch_messages(channel_info, limit)
        except OperationalError as e:
            print(f"Database locked, retrying... Attempt {attempt + 1}/{retries}")
            await asyncio.sleep(5)  # Wait 5 seconds before retrying
            attempt += 1
    raise Exception("Max retries reached, database still locked.")


In [10]:
import asyncio
import time
from telethon.errors import SessionPasswordNeededError
from sqlite3 import OperationalError

async def fetch_with_retry(channel_info, limit=100, retries=3):
    attempt = 0
    while attempt < retries:
        try:
            return await fetch_messages(channel_info, limit)  # Replace with your message-fetching function
        except OperationalError as e:
            print(f"Database is locked, retrying... Attempt {attempt + 1}/{retries}")
            await asyncio.sleep(5)  # Wait 5 seconds before retrying
            attempt += 1
    raise Exception("Max retries reached, database still locked.")


In [ ]:
import pandas as pd
from telethon import TelegramClient
import nest_asyncio
import asyncio
from google.colab import files
import time

# Allow nested event loops for Google Colab
nest_asyncio.apply()

# Telegram API credentials
api_id = '27904749'
api_hash = '5d9f2efaa785a2c4d04b5043088c2813'

# Load the uploaded channels CSV
channels_df = pd.read_csv('channels_to_crawls.csv', header=None)
channel_list = channels_df[0].tolist()  # Use the first column as the channel list

# Initialize Telegram Client
client = TelegramClient(':memory:', api_id, api_hash)

# Retry logic for handling operational errors
def retry_on_lock(func, retries=3, delay=5):
    for _ in range(retries):
        try:
            return func()
        except Exception as e:
            print(f"Error: {e}, retrying in {delay} seconds...")
            time.sleep(delay)
    raise Exception("Retries exhausted.")

# Function to fetch messages from a single channel
async def fetch_messages(channel_username, limit=100):
    messages = []
    try:
        async for message in client.iter_messages(channel_username, limit=limit):
            media_path = None
            if message.media:
                media_path = await message.download_media(file='media/')

            messages.append({
                'channel_username': channel_username,
                'id': message.id,
                'message': message.text,
                'date': message.date,
                'media_path': media_path
            })
    except Exception as e:
        print(f"Error fetching messages from {channel_username}: {e}")
    return messages

# Fetch messages from all channels
async def fetch_from_all_channels():
    all_data = []
    for channel_username in channel_list:
        print(f"Fetching messages from {channel_username}...")
        channel_data = await fetch_messages(channel_username)
        all_data.extend(channel_data)
    return all_data

# Main coroutine
async def main():
    retry_on_lock(lambda: asyncio.run(client.start()))
    async with client:
        all_data = await fetch_from_all_channels()

    # Save data to CSV
    df = pd.DataFrame(all_data)
    output_path = '/content/raw_telegram_data.csv'
    df.to_csv(output_path, index=False)
    files.download(output_path)

    print(df.head())

# Run the main coroutine
asyncio.run(main())


Please enter your phone (or bot token): +251912026082
Please enter the code you received: 88642
Signed in successfully as Hasan Kere Obse; remember to not break the ToS or you will risk an account ban!
Fetching messages from @ZemenExpress...
Fetching messages from @nevacomputer...
Fetching messages from @meneshayeofficial...


Fetching messages from @ethio_brand_collection...
Fetching messages from @Leyueqa...
Fetching messages from @sinayelj...
Fetching messages from @Shewabrand...
Fetching messages from @helloomarketethiopia...
Fetching messages from @modernshoppingcenter...
Fetching messages from @qnashcom...
Fetching messages from @Fashiontera...
Fetching messages from @kuruwear...
Fetching messages from @gebeyaadama...
Fetching messages from @MerttEka...
Fetching messages from @forfreemarket...
Fetching messages from @classybrands...
Fetching messages from @marakibrand...
Fetching messages from @aradabrand2...
Fetching messages from @marakisat2...
Fetching messages from @belaclassic...
Fetching messages from @AwasMart...
Fetching messages from @qnashcom...
Fetching messages from nan...
Error fetching messages from nan: Cannot cast float to any kind of Peer.
Fetching messages from nan...
Error fetching messages from nan: Cannot cast float to any kind of Peer.
Fetching messages from nan...
Error fetching 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  channel_username    id                                            message  \
0    @ZemenExpress  6024                                                      
1    @ZemenExpress  6023                                                      
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                       date                               media_path  
0 2025-01-17 05:06:37+00:00                               media/.jpg  
1 2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg  
2 2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg  
3 2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4  
4 2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg  


In [ ]:
import nltk
nltk.download('punkt')  # Download the punkt tokenizer data
nltk.download('stopwords')  # Download stopwords


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
# Import libraries
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download necessary NLTK resources
nltk.download('punkt')  # Download the punkt tokenizer
nltk.download('stopwords')  # Download stopwords

# Step 1: Load the Dataset
from google.colab import files

# Upload your dataset file
uploaded = files.upload()

# Load the dataset into a DataFrame
dataset = pd.read_csv('raw_telegram_data.csv')

# Display the first few rows
print("Original Dataset:")
print(dataset.head())




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Saving raw_telegram_data.csv to raw_telegram_data.csv
Original Dataset:
  channel_username    id                                            message  \
0    @ZemenExpress  6024                                                NaN   
1    @ZemenExpress  6023                                                NaN   
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date                               media_path  
0  2025-01-17 05:06:37+00:00                               media/.jpg  
1  2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg  
2  2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg  
3  2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4  
4  2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg  


In [15]:
# Step 2: Define Preprocessing Functions
# Function to clean text
def clean_text(text):
    """
    Cleans the given text by removing non-Amharic characters, digits, and extra spaces.
    """
    if pd.isnull(text):  # Handle missing values
        return ''
    # Remove non-Amharic characters and keep spaces
    text = re.sub(r'[^\u1200-\u137F\s]', '', str(text))
    # Remove digits
    text = re.sub(r'\d+', '', text)
    # Normalize multiple spaces to a single space
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Tokenize text
def tokenize_text(text):
    """
    Tokenizes the cleaned text into words using NLTK's word_tokenize.
    """
    if not text:  # Handle empty or null text gracefully
        return []
    return word_tokenize(text)

# Remove stopwords
def remove_stopwords(tokens):
    """
    Removes stopwords from the list of tokens.
    """
    # Define a set of Amharic stopwords for efficient lookup
    amharic_stopwords = set(['ነው', 'እና', 'እርስዎ', 'እንዲሁም', 'በኩል', 'የሚሆነው'])
    return [word for word in tokens if word not in amharic_stopwords]


In [24]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
import pandas as pd

# Dummy function to demonstrate how to label entities (replace this with your actual labeling logic)
def label_in_conll_format(text):
    """
    Function to label tokens (words) in the text. Replace this function with the logic for NER labeling.
    The function returns the tokens in CoNLL format with predefined entity labels.
    """
    # Tokenize the text (simple example, you may need a more sophisticated tokenizer)
    tokens = text.split()

    labeled_text = []

    for token in tokens:
        # Example: Assign labels based on certain conditions (replace with your actual logic)

        # English entity recognition
        if token.lower() in ['product', 'item']:  # Example condition for English product
            label = 'PRODUCT'
        elif token.lower() in ['etb', 'usd']:  # Example: label currencies like ETB, USD as PRICE
            label = 'PRICE'
        elif token.lower() in ['addis', 'ethiopia']:  # Example: label cities/countries as LOCATION
            label = 'LOCATION'

        # Amharic entity recognition (example words, modify as needed)
        elif token in ['እቃ', 'አምስት']:  # 'እቃ' is Amharic for 'product', 'አምስት' is 'five'
            label = 'PRODUCT'
        elif token in ['ብር', 'ኢትቢ'] :  # Amharic currencies (like 'ብር' for Birr, 'ኢትቢ' for ETB)
            label = 'PRICE'
        elif token in ['አዲስ', 'ኢትዮጵያ']:  # 'አዲስ' for Addis Ababa and 'ኢትዮጵያ' for Ethiopia
            label = 'LOCATION'
        else:
            label = 'O'  # O stands for "Other", no entity

        labeled_text.append(f"{token} {label}")

    # Return the labeled text in CoNLL format (each token + label on its own line)
    return "\n".join(labeled_text)

def save_to_conll(df, output_path):
    """
    Function to save labeled data in CoNLL format (each token is labeled in a separate line).
    """
    with open(output_path, 'w') as f:
        for index, row in df.iterrows():
            # Ensure that 'processed_content' column is not empty or NaN before labeling
            if isinstance(row['processed_content'], str) and row['processed_content'].strip():
                labeled_text = label_in_conll_format(row['processed_content'])
                f.write(labeled_text)
                f.write("\n")  # Add an empty line after each sentence

# Assuming you have already preprocessed the data and stored it in 'df'
# Here's an example dataframe (including both English and Amharic)
df = pd.DataFrame({
    'sender': [12345, 67890],
    'timestamp': ['2025-01-16 10:00', '2025-01-16 10:05'],
    'processed_content': ['This is a product costing 100 ETB in Addis Ababa.',
                          'እቃ ሲሆን በ አዲስ ኢትዮጵያ 200 ብር ይዘዋል።']  # Example Amharic message
})

# Save the labeled data to a CoNLL file
output_path = '/content/labeled_telegram_data.conll'  # Save to Colab's environment
save_to_conll(df, output_path)

# Display the file path where the labeled data is saved
print(f"Labeled data saved to {output_path}")


Labeled data saved to /content/labeled_telegram_data.conll


In [17]:
import pandas as pd

# Function to label tokens in CoNLL format (with B- and I- for entities)
def label_in_conll_format(text):
    """
    Function to label tokens (words) in the text. Replace this function with the logic for NER labeling.
    The function returns the tokens in CoNLL format with predefined entity labels.
    """
    tokens = text.split()
    labeled_text = []

    # Variables to track entity labeling (for B- and I- tags)
    current_entity = None  # Keeps track of the current entity
    for i, token in enumerate(tokens):
        # Product Entity Labeling
        if token.lower() in ['baby', 'bottle', 'laptop', 'phone', 'chair']:  # Example products
            if current_entity != 'PRODUCT':
                label = 'B-PRODUCT'  # Start of a new product entity
                current_entity = 'PRODUCT'
            else:
                label = 'I-PRODUCT'  # Inside the product entity
        # Location Entity Labeling
        elif token.lower() in ['addis', 'ababa', 'bole', 'ethiopia']:  # Example locations
            if current_entity != 'LOC':
                label = 'B-LOC'  # Start of a new location entity
                current_entity = 'LOC'
            else:
                label = 'I-LOC'  # Inside the location entity
        # Price Entity Labeling
        elif token.isdigit() or token in ['ብር', 'etb']:  # Example for price (ETB, digits)
            if current_entity != 'PRICE':
                label = 'B-PRICE'  # Start of a new price entity
                current_entity = 'PRICE'
            else:
                label = 'I-PRICE'  # Inside the price entity
        else:
            label = 'O'  # Outside any entity (e.g., common words or punctuation)
            current_entity = None  # Reset entity if no match

        labeled_text.append(f"{token} {label}")

    # Return the labeled text in CoNLL format (each token + label on its own line)
    return "\n".join(labeled_text)

def save_to_conll(df, output_path):
    """
    Function to save labeled data in CoNLL format (each token is labeled in a separate line).
    """
    with open(output_path, 'w') as f:
        for index, row in df.iterrows():
            # Ensure that 'processed_content' column is not empty or NaN before labeling
            if isinstance(row['processed_content'], str) and row['processed_content'].strip():
                labeled_text = label_in_conll_format(row['processed_content'])
                f.write(labeled_text)
                f.write("\n\n")  # Add an empty line after each sentence

# Assuming you have already preprocessed the data and stored it in 'df'
# Here's an example dataframe (including both English and Amharic)
df = pd.DataFrame({
    'sender': [12345, 67890],
    'timestamp': ['2025-01-16 10:00', '2025-01-16 10:05'],
    'processed_content': [
        'This is a baby bottle costing 100 ETB in Addis Ababa.',
        'እቃ ሲሆን በ አዲስ ኢትዮጵያ 200 ብር ይዘዋል።'
    ]  # Example messages in both English and Amharic
})

# Save the labeled data to a CoNLL file
output_path = '/content/labeled_telegram_data.conll'  # Save to Colab's environment
save_to_conll(df, output_path)

# Display the file path where the labeled data is saved
print(f"Labeled data saved to {output_path}")


Labeled data saved to /content/labeled_telegram_data.conll


In [26]:
# Step 4: Separate Metadata and Message Content
# Check column names to ensure correct extraction
if 'channel_username' not in dataset.columns:
    print("Error: 'channel_username' column not found. Please check the column name.")
else:
    structured_data = dataset[['channel_username', 'id', 'date', 'message', 'media_path']]

    # Step 5: Store the preprocessed data in a structured format (CSV)
    output_file = 'structured_data.csv'
    structured_data.to_csv(output_file, index=False)

    # Display the structured data for review
    print("Structured Data Preview:")
    print(structured_data.head())

    # Download the structured data CSV file
    files.download(output_file)


Structured Data Preview:
  channel_username    id                       date  \
0    @ZemenExpress  6024  2025-01-17 05:06:37+00:00   
1    @ZemenExpress  6023  2025-01-17 05:06:37+00:00   
2    @ZemenExpress  6022  2025-01-17 05:06:37+00:00   
3    @ZemenExpress  6021  2025-01-17 05:06:27+00:00   
4    @ZemenExpress  6020  2025-01-16 12:49:25+00:00   

                                             message  \
0                                                NaN   
1                                                NaN   
2  💥💥...................................💥💥\n\n🎯 B...   
3  💥💥...................................💥💥\n\n🎯 B...   
4  💥💥...................................💥💥\n\n📌 R...   

                                media_path  
0                               media/.jpg  
1      media/photo_2025-01-17_05-06-37.jpg  
2  media/photo_2025-01-17_05-06-37 (1).jpg  
3   media/document_2025-01-17_05-06-27.mp4  
4      media/photo_2025-01-16_12-49-25.jpg  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
print(dataset.columns)


Index(['channel_username', 'id', 'message', 'date', 'media_path'], dtype='object')


In [28]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize

# Download the correct punkt tokenizer and all languages support
nltk.download('punkt')
nltk.download('punkt_tab')  # This downloads the tab-specific tokenizer

# Load your dataset (assuming it's already loaded in `dataset`)
dataset = pd.read_csv("raw_telegram_data.csv")

# Extract a subset of the dataset for labeling
subset = dataset['message'].head(10)  # Adjust the number of rows based on the subset you need

# Example of entity labels (expand these as needed)
product_keywords = ['አፕል', 'ስልኩ', 'መኪና']  # Example product-related keywords in Amharic
location_keywords = ['ኢትዮጵያ', 'አዲስ', 'ተማ']  # Example location-related keywords in Amharic

# Function to label the entities
def label_entities(text):
    if pd.isnull(text):  # Check for NaN or None values
        return ""

    tokens = word_tokenize(str(text))  # Convert to string in case it's float or other type
    labeled_tokens = []

    for token in tokens:
        label = 'O'  # Default label is 'O' (for outside any entity)

        # Check for product-related words
        if any(keyword in token for keyword in product_keywords):
            label = 'B-PRODUCT'
        # Check for location-related words
        elif any(keyword in token for keyword in location_keywords):
            label = 'B-LOCATION'

        # Add the token and its label to the list
        labeled_tokens.append(f"{token} {label}")

    return '\n'.join(labeled_tokens)

# Apply the labeling function to the subset of messages
labeled_data = subset.apply(label_entities)

# Format the CoNLL output: Each message gets separated by a blank line
conll_output = '\n\n'.join(labeled_data)

# Write to a file
with open("labeled_data.conll", 'w', encoding='utf-8') as f:
    f.write(conll_output)

# Print out the labeled data preview
print("CoNLL Labeled Data Preview:")
print(conll_output)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


CoNLL Labeled Data Preview:




💥💥 O
................................... O
💥💥 O
🎯 O
Baby O
Feeding O
Bottle O
Spoon O
with O
Container O
Baby O
Food O
Feeder O
💯 O
High O
Quality O
🍼የህፃናት O
መመገቢያ O
ጡጦ O
👍Silicone O
Squeeze O
Spoon O
Feeder O
for O
Infant O
Food O
Dispensing O
and O
Feeding O
👍Safe O
Material O
👍Filterable O
Food O
👍Strong O
Sealing O
👍Easy O
to O
Use O
ዋጋ፦ O
💵🏷 O
400 O
ብር O
♦️ውስን O
ፍሬ O
ነው O
ያለው O
🔥🔥🔥 O
🏢 O
አድራሻ👉 O
📍ቁ.1️⃣♦️መገናኛ O
መሰረት O
ደፋር O
ሞል O
ሁለተኛ O
ፎቅ O
ቢሮ O
ቁ. O
S05/S06 O
📍 O
ቁ.2️⃣♦️ፒያሳ O
ጊዮርጊስ O
አደባባይ O
ራመት_ታቦር_ኦዳ_ህንፃ O
1ኛ O
ፎቅ O
ሱቅ O
ቁ. O
G1 O
-107 O
💧💧💧💧 O
📲 O
0902660722 O
📲 O
0928460606 O
➡️ O
🔴ፒያሳ O
ቅርንጫፍ O
0941337070 O
🔖 O
💬በTelegram O
ለማዘዝ O
⤵️ O
ይጠቀሙ🔽 O
@ O
zemencallcenter O
@ O
zemenexpressadmin O
ለተጨማሪ O
ማብራሪያ O
የቴሌግራም O
ገፃችን⤵️ O
https O
: O
//telegram.me/zemenexpress O
💥💥 O
................................... O
💥💥 O

💥💥 O
................................... O
💥💥 O
🎯 O
Baby O
Feeding O
Bottle O
Spoon O
with O
Container O
Baby O
Food O
Feeder O
💯 O
High

In [29]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')  # To ensure compatibility for tokenization

# Example keywords for Product, Location, and Price in Amharic (expand this list based on your needs)
product_keywords = ['ስልኩ', 'አፕል', 'መኪና', 'ቡና', 'በረም']
location_keywords = ['አዲስ', 'ተማ', 'ኢትዮጵያ', 'ቦሌ', 'ሐምሌ']
price_keywords = ['ዋጋ', 'ብር', 'እትም', 'በ', '100']

# Load dataset
dataset = pd.read_csv("raw_telegram_data.csv")

# Extract a subset of the dataset for labeling (30-50 messages)
subset = dataset['message'].head(50)  # Adjust the number of rows based on the subset size

# Function to label the entities (Product, Location, Price)
def label_entities(text):
    if pd.isnull(text):  # Check for NaN values
        return ""

    tokens = word_tokenize(str(text))  # Ensure the text is in string format
    labeled_tokens = []

    prev_label = None  # To handle the continuity of the entity

    for token in tokens:
        label = 'O'  # Default label is 'O' (outside any entity)

        # Check for product-related words
        if any(keyword in token for keyword in product_keywords):
            if prev_label != 'B-PRODUCT':
                label = 'B-PRODUCT'  # Beginning of a product entity
            else:
                label = 'I-PRODUCT'  # Inside a product entity
            prev_label = 'B-PRODUCT' if label == 'B-PRODUCT' else 'I-PRODUCT'

        # Check for location-related words
        elif any(keyword in token for keyword in location_keywords):
            if prev_label != 'B-LOC':
                label = 'B-LOC'  # Beginning of a location entity
            else:
                label = 'I-LOC'  # Inside a location entity
            prev_label = 'B-LOC' if label == 'B-LOC' else 'I-LOC'

        # Check for price-related words
        elif any(keyword in token for keyword in price_keywords):
            if prev_label != 'B-PRICE':
                label = 'B-PRICE'  # Beginning of a price entity
            else:
                label = 'I-PRICE'  # Inside a price entity
            prev_label = 'B-PRICE' if label == 'B-PRICE' else 'I-PRICE'

        # Add the token and its label to the list
        labeled_tokens.append(f"{token} {label}")

    return '\n'.join(labeled_tokens)

# Apply the labeling function to the subset of messages
labeled_data = subset.apply(label_entities)

# Format the CoNLL output: Each message gets separated by a blank line
conll_output = '\n\n'.join(labeled_data)

# Write the labeled data to a CoNLL format file
with open("labeled_data.conll", 'w', encoding='utf-8') as f:
    f.write(conll_output)

# Print out the labeled data preview
print("CoNLL Labeled Data Preview:")
print(conll_output)


CoNLL Labeled Data Preview:




💥💥 O
................................... O
💥💥 O
🎯 O
Baby O
Feeding O
Bottle O
Spoon O
with O
Container O
Baby O
Food O
Feeder O
💯 O
High O
Quality O
🍼የህፃናት O
መመገቢያ O
ጡጦ O
👍Silicone O
Squeeze O
Spoon O
Feeder O
for O
Infant O
Food O
Dispensing O
and O
Feeding O
👍Safe O
Material O
👍Filterable O
Food O
👍Strong O
Sealing O
👍Easy O
to O
Use O
ዋጋ፦ B-PRICE
💵🏷 O
400 O
ብር I-PRICE
♦️ውስን O
ፍሬ O
ነው O
ያለው O
🔥🔥🔥 O
🏢 O
አድራሻ👉 O
📍ቁ.1️⃣♦️መገናኛ O
መሰረት O
ደፋር O
ሞል O
ሁለተኛ O
ፎቅ O
ቢሮ O
ቁ. O
S05/S06 O
📍 O
ቁ.2️⃣♦️ፒያሳ O
ጊዮርጊስ O
አደባባይ O
ራመት_ታቦር_ኦዳ_ህንፃ O
1ኛ O
ፎቅ O
ሱቅ O
ቁ. O
G1 O
-107 O
💧💧💧💧 O
📲 O
0902660722 O
📲 O
0928460606 O
➡️ O
🔴ፒያሳ O
ቅርንጫፍ O
0941337070 O
🔖 O
💬በTelegram B-PRICE
ለማዘዝ O
⤵️ O
ይጠቀሙ🔽 O
@ O
zemencallcenter O
@ O
zemenexpressadmin O
ለተጨማሪ O
ማብራሪያ O
የቴሌግራም O
ገፃችን⤵️ O
https O
: O
//telegram.me/zemenexpress O
💥💥 O
................................... O
💥💥 O

💥💥 O
................................... O
💥💥 O
🎯 O
Baby O
Feeding O
Bottle O
Spoon O
with O
Container O
Baby O
Food O

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving raw_telegram_data.csv to raw_telegram_data (2).csv


In [30]:
import pandas as pd
from datasets import Dataset

# Load the uploaded CSV file
df = pd.read_csv("raw_telegram_data.csv")

# Convert the pandas DataFrame into a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Check the dataset structure
print(dataset)


Dataset({
    features: ['channel_username', 'id', 'message', 'date', 'media_path'],
    num_rows: 2200
})


In [31]:
import re

# Define the preprocess_text function
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove user mentions (e.g., @username)
    text = re.sub(r'@\w+', '', text)

    # Remove hashtags
    text = re.sub(r'#\w+', '', text)

    # Remove special characters, numbers, and punctuations except letters and spaces
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [32]:
import pandas as pd
from datasets import Dataset
import re
import nltk
from transformers import XLMRobertaTokenizer

# Download required resources
nltk.download("punkt")

# Assuming the dataset is uploaded, let's load it into a pandas DataFrame
df = pd.read_csv("raw_telegram_data.csv")

# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Initialize the tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

# Define text preprocessing functions
def preprocess_text(text):
    """Preprocess text: remove unwanted characters, tokenize, etc."""
    text = re.sub(r'[^\u1200-\u137F\s]', '', str(text))  # Keep only Amharic characters
    return text

# Preprocess the 'message' column
df['cleaned_message'] = df['message'].apply(preprocess_text)

# Now proceed with labeling and fine-tuning...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [33]:
import pandas as pd
from datasets import Dataset
import re
import nltk
from transformers import XLMRobertaTokenizer

# Download required resources for NLTK
nltk.download("punkt")

# Assuming the dataset is uploaded, let's load it into a pandas DataFrame
df = pd.read_csv("raw_telegram_data.csv")  # Replace with your dataset path

# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Initialize the tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

# Define text preprocessing functions
def preprocess_text(text):
    """Preprocess text: remove unwanted characters, tokenize, etc."""
    text = re.sub(r'[^\u1200-\u137F\s]', '', str(text))  # Keep only Amharic characters
    return text

# Convert the Hugging Face Dataset back to pandas DataFrame for easier manipulation
df = pd.DataFrame(dataset)

# Preprocess the 'message' column
df['cleaned_message'] = df['message'].apply(preprocess_text)

# Now proceed with labeling and fine-tuning...
print(df.head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  channel_username    id                                            message  \
0    @ZemenExpress  6024                                               None   
1    @ZemenExpress  6023                                               None   
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date                               media_path  \
0  2025-01-17 05:06:37+00:00                               media/.jpg   
1  2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg   
2  2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg   
3  2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4   
4  2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg   

                                     cleaned_message  
0                              

In [36]:
from transformers import XLMRobertaForTokenClassification, XLMRobertaTokenizer, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import evaluate  # Use evaluate for metrics
import torch
from sklearn.model_selection import train_test_split
import pandas as pd

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cpu


In [ ]:
from transformers import XLMRobertaForTokenClassification, XLMRobertaTokenizer

# Load the pretrained XLM-Roberta model for token classification (NER task)
model = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base")

# Load the tokenizer for XLM-Roberta
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
import pandas as pd

# Load the raw telegram dataset
df = pd.read_csv('raw_telegram_data.csv')

# Preview the dataset
df.head()


,channel_username,id,message,date,media_path
0,@ZemenExpress,6024,NaN,2025-01-17 05:06:37+00:00,media/.jpg
1,@ZemenExpress,6023,NaN,2025-01-17 05:06:37+00:00,media/photo_2025-01-17_05-06-37.jpg
2,@ZemenExpress,6022,💥💥...................................💥💥\n\n🎯 B...,2025-01-17 05:06:37+00:00,media/photo_2025-01-17_05-06-37 (1).jpg
3,@ZemenExpress,6021,💥💥...................................💥💥\n\n🎯 B...,2025-01-17 05:06:27+00:00,media/document_2025-01-17_05-06-27.mp4
4,@ZemenExpress,6020,💥💥...................................💥💥\n\n📌 R...,2025-01-16 12:49:25+00:00,media/photo_2025-01-16_12-49-25.jpg


In [38]:
df['message'] = df['message'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)



In [39]:
# Replace NaN values with an empty string or a placeholder
df['message'] = df['message'].fillna('UNKNOWN')

# If you prefer a placeholder, you can use:
# df['message'] = df['message'].fillna('UNKNOWN')


In [ ]:
# Drop rows where 'message' column contains NaN values
df = df.dropna(subset=['message'])

# Verify the result
print(df.head())


  channel_username    id                                            message  \
0    @ZemenExpress  6024                                            UNKNOWN   
1    @ZemenExpress  6023                                            UNKNOWN   
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date                               media_path  
0  2025-01-17 05:06:37+00:00                               media/.jpg  
1  2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg  
2  2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg  
3  2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4  
4  2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg  


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('raw_telegram_data.csv')

# Check the first few rows of the dataset
print(df.head())


  channel_username    id                                            message  \
0    @ZemenExpress  6024                                                NaN   
1    @ZemenExpress  6023                                                NaN   
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date                               media_path  
0  2025-01-17 05:06:37+00:00                               media/.jpg  
1  2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg  
2  2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg  
3  2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4  
4  2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg  


In [40]:
# Replace NaN values in 'message' column with empty strings
df['message'] = df['message'].fillna('')

# Alternatively, you can drop rows with NaN in the 'message' column:
# df = df.dropna(subset=['message'])

# Check the cleaned data
print(df.head())


  channel_username    id                                            message  \
0    @ZemenExpress  6024                                            UNKNOWN   
1    @ZemenExpress  6023                                            UNKNOWN   
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date                               media_path  
0  2025-01-17 05:06:37+00:00                               media/.jpg  
1  2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg  
2  2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg  
3  2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4  
4  2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg  


In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForTokenClassification

# Load the pre-trained XLM-Roberta model for token classification (NER)
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaForTokenClassification.from_pretrained('xlm-roberta-base')


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
import pandas as pd

# Load your dataset (replace 'raw_telegram_data.csv' with the actual file path)
df = pd.read_csv('raw_telegram_data.csv')

# Check the first few rows to understand the structure of your dataset
print(df.head())


  channel_username    id                                            message  \
0    @ZemenExpress  6024                                                NaN   
1    @ZemenExpress  6023                                                NaN   
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date                               media_path  
0  2025-01-17 05:06:37+00:00                               media/.jpg  
1  2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg  
2  2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg  
3  2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4  
4  2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg  


In [42]:
from transformers import XLMRobertaTokenizer, XLMRobertaForTokenClassification

# Load the pre-trained XLM-Roberta model for token classification (NER)
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaForTokenClassification.from_pretrained('xlm-roberta-base')

# Example: Tokenizing and preparing data
text = "ስም የእቃ ስም እባክህ ምን ይኸው የእቃ ተመን?"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
print(inputs)

# You can fine-tune the model as shown earlier using the Hugging Face Trainer.




model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[     0,  37851,  30179,   6550,  37851,   4708,   5657,   4133,   4359,
          14719, 220555,  30179,   6550, 131253,    548,     32,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [43]:
import pandas as pd
from datasets import Dataset
from transformers import XLMRobertaTokenizer

# Load your labeled data (in CoNLL format)
def load_conll(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.read().strip().split('\n\n')

    sentences = []
    labels = []

    for line in lines:
        sentence = []
        label = []
        for word in line.splitlines():
            parts = word.split()
            if len(parts) == 2:
                sentence.append(parts[0])
                label.append(parts[1])
        sentences.append(sentence)
        labels.append(label)

    return sentences, labels

# Load your dataset
sentences, labels = load_conll('labeled_data.conll')

# Create a DataFrame
df = pd.DataFrame({
    'tokens': sentences,
    'labels': labels
})

# Convert DataFrame to Hugging Face Dataset format
dataset = Dataset.from_pandas(df)


In [ ]:
# Inspect the first example in the dataset
print(dataset[0])  # This will print the first record (tokens and labels)


{'tokens': ['💥💥', '...................................', '💥💥', '🎯', 'Baby', 'Feeding', 'Bottle', 'Spoon', 'with', 'Container', 'Baby', 'Food', 'Feeder', '💯', 'High', 'Quality', '🍼የህፃናት', 'መመገቢያ', 'ጡጦ', '👍Silicone', 'Squeeze', 'Spoon', 'Feeder', 'for', 'Infant', 'Food', 'Dispensing', 'and', 'Feeding', '👍Safe', 'Material', '👍Filterable', 'Food', '👍Strong', 'Sealing', '👍Easy', 'to', 'Use', 'ዋጋ፦', '💵🏷', '400', 'ብር', '♦️ውስን', 'ፍሬ', 'ነው', 'ያለው', '🔥🔥🔥', '🏢', 'አድራሻ👉', '📍ቁ.1️⃣♦️መገናኛ', 'መሰረት', 'ደፋር', 'ሞል', 'ሁለተኛ', 'ፎቅ', 'ቢሮ', 'ቁ.', 'S05/S06', '📍', 'ቁ.2️⃣♦️ፒያሳ', 'ጊዮርጊስ', 'አደባባይ', 'ራመት_ታቦር_ኦዳ_ህንፃ', '1ኛ', 'ፎቅ', 'ሱቅ', 'ቁ.', 'G1', '-107', '💧💧💧💧', '📲', '0902660722', '📲', '0928460606', '➡️', '🔴ፒያሳ', 'ቅርንጫፍ', '0941337070', '🔖', '💬በTelegram', 'ለማዘዝ', '⤵️', 'ይጠቀሙ🔽', '@', 'zemencallcenter', '@', 'zemenexpressadmin', 'ለተጨማሪ', 'ማብራሪያ', 'የቴሌግራም', 'ገፃችን⤵️', 'https', ':', '//telegram.me/zemenexpress', '💥💥', '...................................', '💥💥'], 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [44]:
def tokenize_and_align_labels(examples):
    # Ensure that sentences are properly tokenized
    tokenized_inputs = tokenizer(examples['tokens'], padding=True, truncation=True, is_split_into_words=True)

    # Ensure labels align with the tokenized sentences
    labels = examples['labels']
    new_labels = []
    for i, word_ids in enumerate(tokenized_inputs.word_ids(batch_index=0)):
        if word_ids is None:
            new_labels.append(-100)  # Special tokens (like [CLS], [SEP]) are ignored
        else:
            new_labels.append(labels[word_ids])

    tokenized_inputs['labels'] = new_labels
    return tokenized_inputs


In [ ]:
# Remove empty sentences or rows that don't have tokens or labels
dataset = dataset.filter(lambda example: len(example['tokens']) > 0 and len(example['labels']) > 0)


Filter:   0%|          | 0/48 [00:00<?, ? examples/s]

In [45]:
from transformers import XLMRobertaTokenizerFast

# Load the fast tokenizer for XLM-Roberta
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')


In [48]:
print(tokenizer.is_fast)  # Should return True


True


In [49]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base", use_fast=True)
print(tokenizer.is_fast)  # This should return True



True


In [50]:
from transformers import AutoTokenizer, XLMRobertaForTokenClassification

# Ensure the correct tokenizer is loaded
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base", use_fast=True)
print("Is Fast Tokenizer:", tokenizer.is_fast)  # Should return True

# Load the model
model = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=9)


Is Fast Tokenizer: True


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
from transformers import AutoTokenizer, XLMRobertaForTokenClassification
import transformers

# Check library version
print("Transformers version:", transformers.__version__)

# Use AutoTokenizer with use_fast=True
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base", use_fast=True)
print("Is Fast Tokenizer:", tokenizer.is_fast)  # Should return True

# Load model for token classification
model = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=9)

# Example input for prediction
input_text = ["Hello", "my", "name", "is", "John"]

# Tokenize input
inputs = tokenizer(input_text, is_split_into_words=True, return_tensors="pt", padding=True, truncation=True)
print(inputs)

# Predict using the model
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = logits.argmax(dim=-1)

print("Predictions:", predictions)


Transformers version: 4.47.1
Is Fast Tokenizer: True


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[    0, 35378,   759,  9351,    83,  4939,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
Predictions: tensor([[6, 6, 6, 6, 6, 6, 6]])


In [52]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "xlm-roberta-large-finetuned-conll03-english"  # A pre-trained NER model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [53]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Load the pre-trained NER model
model_name = "xlm-roberta-large-finetuned-conll03-english"  # Example pre-trained NER model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Load your dataset (replace with your file path if needed)
df = pd.read_csv("raw_telegram_data.csv")

# Display the first few rows of the CSV file to ensure it's loaded correctly
print(df.head())

# Extract the column containing text messages (replace 'message' with the actual column name if different)
messages = df['message'].dropna().tolist()  # Handle missing values if any

# Loop through each message and make predictions
for message in messages:
    # Tokenize the message
    inputs = tokenizer(message.split(), is_split_into_words=True, return_tensors="pt", padding=True, truncation=True)

    # Predict
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = logits.argmax(dim=-1)

    # Map predicted indices to label names
    label_ids = predictions[0].tolist()  # Get predictions for the first (and only) sentence
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])  # Convert input IDs back to tokens
    labels = [model.config.id2label[label_id] for label_id in label_ids]  # Map indices to labels

    # Print the tokens and their corresponding predicted labels for the current message
    print(f"Message: {message}")
    for token, label in zip(tokens, labels):
        print(f"Token: {token}, Label: {label}")
    print("\n" + "-"*50 + "\n")


Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Streaming output truncated to the last 5000 lines.
Token: ቾ, Label: O
Token: ት, Label: O
Token: ▁እና, Label: O
Token: ▁በደ, Label: O
Token: ህን, Label: O
Token: ነት, Label: O
Token: ▁ለ, Label: O
Token: ረጅም, Label: O
Token: ▁ሰአት, Label: O
Token: ▁የሚ, Label: O
Token: ጠቀሙ, Label: O
Token: በት, Label: O
Token: ▁, Label: O
Token: <unk>, Label: O
Token: ለ, Label: O
Token: ቢ, Label: O
Token: ሮ, Label: O
Token: ▁እና, Label: O
Token: ▁, Label: O
Token: ለም, Label: O
Token: ግብ, Label: O
Token: ▁ቤት, Label: O
Token: ▁ለተ, Label: O
Token: ለያ, Label: O
Token: ዬ, Label: O
Token: ▁አ, Label: O
Token: ገል, Label: O
Token: ግ, Label: O
Token: ሎ, Label: O
Token: ቶች, Label: O
Token: ▁መጠቀም, Label: O
Token: ▁ይችላሉ, Label: O
Token: ▁, Label: O
Token: <unk>, Label: O
Token: ም, Label: O
Token: ስ, Label: O
Token: ሉ, Label: O
Token: ▁የ, Label: O
Token: ፈለጉ, Label: O
Token: ትን, Label: O
Token: ▁ያህል, Label: O
Token: ▁መቆ, Label: O
Token: የት, Label: O
Token: ▁የሚችል, Label: O
Token: ▁ሲሆን, Label: O
Token: ▁አንድ, Label: O
Token: ▁ቁል

In [54]:
def encode_dataset(df):
    # Check if the messages are strings and convert to a list of strings (if needed)
    messages = df['message'].tolist()  # Convert to list if it's not already a list

    # Tokenize the messages (input must be a list of strings)
    encodings = tokenizer(messages, truncation=True, padding=True, max_length=512, is_split_into_words=False)

    # Assuming 'ner_labels' is a column with token-level entity labels for each message
    labels = []
    for i in range(len(df)):
        label = df['ner_labels'].iloc[i]  # Adjust this line if your labels are in a different format
        labels.append(label)  # This should be a list of token-level labels

    encodings['labels'] = labels
    return encodings


In [59]:
from google.colab import auth
auth.authenticate_user()

# To set it as a secret token, use the following code in Colab:
# from huggingface_hub import login
# login("your_token_here")


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("raw_telegram_data.csv")

# Inspect the dataset
print(df.head())

# Example of what the data might look like:
# message: "John went to Paris"
# ner_labels: "B-PER O O B-LOC"

# We will assume 'ner_labels' contains the space-separated entity tags for each token in the message.


  channel_username    id                                            message  \
0    @ZemenExpress  6024                                                NaN   
1    @ZemenExpress  6023                                                NaN   
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date                               media_path  
0  2025-01-17 05:06:37+00:00                               media/.jpg  
1  2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg  
2  2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg  
3  2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4  
4  2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg  


In [60]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('raw_telegram_data.csv')

# Check the first few rows to understand the structure
print(df.head())

# Handle missing messages: Replace NaN or empty messages with a placeholder (e.g., "No content")
df['message'] = df['message'].fillna('No content')

# Define the tokenizer (e.g., XLM-Roberta for multilingual support)
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")

# Function to tokenize the message
def tokenize_and_align_labels(examples):
    # Tokenize the text (assuming 'message' contains the actual text)
    tokenized_inputs = tokenizer(examples['message'], truncation=True, padding=True, max_length=512)

    # If you have 'ner_labels' column, you would align it with tokenized inputs
    # For now, assume that labels are 'O' (for simplicity) if there is no label column
    labels = ['O'] * len(tokenized_inputs['input_ids'])  # Dummy label, update with actual logic if you have labels

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

# Convert the dataset to Hugging Face's Dataset format
dataset = Dataset.from_pandas(df)

# Apply the tokenization
dataset = dataset.map(tokenize_and_align_labels, batched=True)

# Split the dataset into training and testing (80%/20% split for example)
train_dataset, eval_dataset = dataset.train_test_split(test_size=0.2).values()

# Define the model (using XLM-Roberta for token classification)
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-large", num_labels=2)  # Change num_labels as needed





  channel_username    id                                            message  \
0    @ZemenExpress  6024                                                NaN   
1    @ZemenExpress  6023                                                NaN   
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date                               media_path  
0  2025-01-17 05:06:37+00:00                               media/.jpg  
1  2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg  
2  2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg  
3  2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4  
4  2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg  


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Add a placeholder 'ner_labels' column with dummy labels
# Example of labels for: "John lives in New York"
# 'B-PER' = Beginning of person, 'O' = Outside entity, 'B-LOC' = Beginning of location
df['ner_labels'] = [["B-PER", "O", "O", "B-LOC", "I-LOC"]] * len(df)

print(df.head())


  channel_username    id                                            message  \
0    @ZemenExpress  6024                                                NaN   
1    @ZemenExpress  6023                                                NaN   
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date                               media_path  \
0  2025-01-17 05:06:37+00:00                               media/.jpg   
1  2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg   
2  2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg   
3  2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4   
4  2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg   

                    ner_labels  
0  [B-PER, O, O, B-LOC, I-LOC]  
1  [B-PER, O, O, B-L

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
file_path = "raw_telegram_data.csv"
df = pd.read_csv(file_path)

# Add dummy NER labels (replace this with actual labels if available)
df['ner_labels'] = [["B-PER", "O", "O", "B-LOC", "I-LOC"]] * len(df)  # Example labels

# Split the dataset into training, validation, and test sets
train_data, temp_data = train_test_split(df, test_size=0.3, random_state=42)  # 70% train
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)  # 15% val, 15% test


In [ ]:
from datasets import Dataset, DatasetDict

# Convert pandas DataFrame to datasets.Dataset
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

# Create DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

print(dataset)


DatasetDict({
    train: Dataset({
        features: ['channel_username', 'id', 'message', 'date', 'media_path', 'ner_labels', '__index_level_0__'],
        num_rows: 1540
    })
    validation: Dataset({
        features: ['channel_username', 'id', 'message', 'date', 'media_path', 'ner_labels', '__index_level_0__'],
        num_rows: 330
    })
    test: Dataset({
        features: ['channel_username', 'id', 'message', 'date', 'media_path', 'ner_labels', '__index_level_0__'],
        num_rows: 330
    })
})


In [ ]:
def tokenize_and_align_labels(example):
    # Ensure 'message' is a list of strings even when batched
    texts = example["message"] if isinstance(example["message"], list) else [example["message"]]

    # Tokenize the input text
    tokenized_inputs = tokenizer(
        texts,  # Ensure this is a list of strings (one or more)
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"  # Ensure PyTorch tensors are returned
    )

    # Map NER labels to integers (assumed to be in 'ner_labels')
    label_map = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-LOC': 3, 'I-LOC': 4}
    labels = example["ner_labels"]

    # Ensure labels are a list and align them to tokenized inputs
    if not isinstance(labels, list):
        labels = []

    encoded_labels = [label_map.get(label, 0) for label in labels]
    encoded_labels = encoded_labels[:512] + [0] * (512 - len(encoded_labels))
    tokenized_inputs["labels"] = encoded_labels

    return tokenized_inputs


In [ ]:
# Check the format of 'message' and 'ner_labels' in your dataset
print(dataset["train"][0]["message"])  # Ensure this is a string or list of strings
print(dataset["train"][0]["ner_labels"])  # Ensure this is a list of labels


100% ንፁ ቆዳ የተሰሩ ጫማዎች
🇪🇹 የሀገራችን ምርት 🇪🇹
     🔥2700 ብር ብቻ🔥
☎️ 0911871330
🏢ቦሌ ኦሎምፒያ ጌቱ ኮሜርሻል አጠገብ ራዩማ ህንፃ 119 ቁጥር
             ኩሩ ጫማ
የቴሌግራም አባል ይሁኑ T.me/kuruwear
👉 Code 511
['B-PER', 'O', 'O', 'B-LOC', 'I-LOC']


In [ ]:
def tokenize_and_align_labels(batch):
    # Tokenize the batch of messages (ensure 'message' is a list of strings)
    texts = batch["message"]

    # Tokenize the input text batch (ensure the texts are in the correct format)
    tokenized_inputs = tokenizer(
        texts,  # This should be a list of strings
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",  # Ensure PyTorch tensors are returned
    )

    # Label map
    label_map = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-LOC': 3, 'I-LOC': 4}

    # Convert the NER labels for the batch into indices
    labels = batch["ner_labels"]
    encoded_labels = []

    for label_list in labels:
        # Map labels to indices (ensure label_list is a list of strings)
        label_ids = [label_map.get(label, 0) for label in label_list]
        # Pad or truncate the labels to match the tokenized input length
        label_ids = label_ids[:512] + [0] * (512 - len(label_ids))
        encoded_labels.append(label_ids)

    tokenized_inputs["labels"] = encoded_labels  # Add the labels to the tokenized input
    return tokenized_inputs


In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForTokenClassification

# Define the number of labels (adjust this as per your task)
num_labels = 4  # Example: 4 labels (O, PER, LOC, ORG)

# Initialize the XLM-Roberta tokenizer and model
xlm_roberta_tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-large")
xlm_roberta_model = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-large", num_labels=num_labels)


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define your tokenization function (refer to the previous response)
def tokenize_and_align_labels(batch):
    texts = batch['message']
    tokenized_inputs = tokenizer(texts, padding='max_length', truncation=True, max_length=512, return_tensors="pt")
    labels = batch['ner_labels']
    # Align the labels (make sure to handle them as per your dataset format)
    # Add padding/truncation to labels
    tokenized_inputs['labels'] = labels  # Your label preparation code
    return tokenized_inputs


In [ ]:
# Check for missing data
print(df.isnull().sum())

# Reset index to avoid inconsistencies
df = df.reset_index(drop=True)

# Verify column names
print(df.columns)


channel_username      0
id                    0
message             834
date                  0
media_path           25
ner_labels            0
dtype: int64
Index(['channel_username', 'id', 'message', 'date', 'media_path',
       'ner_labels'],
      dtype='object')


In [ ]:
from google.colab import files

uploaded = files.upload()  # Opens a dialog to upload files


Saving raw_telegram_data.csv to raw_telegram_data.csv


In [ ]:
import pandas as pd
from datasets import Dataset

# Load your dataset
df = pd.read_csv('raw_telegram_data.csv')

# Ensure the dataset has 'tokens' and 'labels' columns
# If not, preprocess accordingly

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)


In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv('raw_telegram_data.csv')
print(data.head())


  channel_username    id                                            message  \
0    @ZemenExpress  6024                                                NaN   
1    @ZemenExpress  6023                                                NaN   
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date                               media_path  
0  2025-01-17 05:06:37+00:00                               media/.jpg  
1  2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg  
2  2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg  
3  2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4  
4  2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg  


In [66]:
import pandas as pd

# Load the dataset
data = pd.read_csv("raw_telegram_data.csv")

# Display basic info and head of the dataset
print(data.info())
print(data.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   channel_username  2200 non-null   object
 1   id                2200 non-null   int64 
 2   message           1366 non-null   object
 3   date              2200 non-null   object
 4   media_path        2175 non-null   object
dtypes: int64(1), object(4)
memory usage: 86.1+ KB
None
  channel_username    id                                            message  \
0    @ZemenExpress  6024                                                NaN   
1    @ZemenExpress  6023                                                NaN   
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date             

In [67]:
# Drop rows with NaN messages
data = data.dropna(subset=['message'])

# Optional: Clean up noisy characters (e.g., emojis)
import re
data['cleaned_message'] = data['message'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Tokenize messages
data['tokens'] = data['cleaned_message'].apply(lambda x: x.split())

# Placeholder labels for tokens (use "O" for non-entities)
data['labels'] = data['tokens'].apply(lambda tokens: ["O"] * len(tokens))

# Display a sample row
print(data[['tokens', 'labels']].head())


                                              tokens  \
2  [Baby, Feeding, Bottle, Spoon, with, Container...   
3  [Baby, Feeding, Bottle, Spoon, with, Container...   
4  [Replacement, Silicon, Nipple, የጡት, ጫፍ, ዋጋ, 40...   
8  [2in1, Glass, oil, Sprayer, and, Dispenser, ከመ...   
9  [2in1, Glass, oil, Sprayer, and, Dispenser, ከመ...   

                                              labels  
2  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  
3  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  
4  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  
8  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  
9  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  


<ipython-input-67-c89530fc4364>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cleaned_message'] = data['message'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
<ipython-input-67-c89530fc4364>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tokens'] = data['cleaned_message'].apply(lambda x: x.split())
<ipython-input-67-c89530fc4364>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [68]:
# Generate "O" labels for each token in the dataset
data['labels'] = data['tokens'].apply(lambda tokens: ["O"] * len(tokens))

# Verify the structure
print(data[['tokens', 'labels']].head())


                                              tokens  \
2  [Baby, Feeding, Bottle, Spoon, with, Container...   
3  [Baby, Feeding, Bottle, Spoon, with, Container...   
4  [Replacement, Silicon, Nipple, የጡት, ጫፍ, ዋጋ, 40...   
8  [2in1, Glass, oil, Sprayer, and, Dispenser, ከመ...   
9  [2in1, Glass, oil, Sprayer, and, Dispenser, ከመ...   

                                              labels  
2  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  
3  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  
4  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  
8  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  
9  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  


In [69]:
# Check for mismatched lengths
mismatch = data[data['tokens'].str.len() != data['labels'].str.len()]
if not mismatch.empty:
    print("Mismatched rows found:", mismatch)
else:
    print("All rows are consistent!")


All rows are consistent!


In [70]:
# Generate tokens and placeholder labels
data['tokens'] = data['cleaned_message'].apply(lambda x: x.split())
data['labels'] = data['tokens'].apply(lambda tokens: ["O"] * len(tokens))

# Verify structure
print(data[['tokens', 'labels']].head())

# Check for mismatched lengths
mismatch = data[data['tokens'].str.len() != data['labels'].str.len()]
if not mismatch.empty:
    print("Mismatched rows found:", mismatch)
else:
    print("All rows are consistent!")


                                              tokens  \
2  [Baby, Feeding, Bottle, Spoon, with, Container...   
3  [Baby, Feeding, Bottle, Spoon, with, Container...   
4  [Replacement, Silicon, Nipple, የጡት, ጫፍ, ዋጋ, 40...   
8  [2in1, Glass, oil, Sprayer, and, Dispenser, ከመ...   
9  [2in1, Glass, oil, Sprayer, and, Dispenser, ከመ...   

                                              labels  
2  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  
3  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  
4  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  
8  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  
9  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  
All rows are consistent!


In [71]:
# Format as list of dictionaries for Hugging Face
formatted_data = [
    {"tokens": tokens, "labels": labels}
    for tokens, labels in zip(data['tokens'], data['labels'])
]

# Split into train and validation sets
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(formatted_data, test_size=0.2, random_state=42)


In [72]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

def tokenize_and_align_labels(data):
    tokenized_inputs = tokenizer(
        [example["tokens"] for example in data],
        is_split_into_words=True,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )

    labels = []
    for i, example in enumerate(data):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label_map[example["labels"][word_id]])
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Map labels to IDs
unique_labels = set(label for example in train_data for label in example['labels'])
label_map = {label: i for i, label in enumerate(unique_labels)}

tokenized_train = tokenize_and_align_labels(train_data)
tokenized_val = tokenize_and_align_labels(val_data)


In [73]:
from datasets import Dataset

# Convert the tokenized data into Dataset format
train_dataset = Dataset.from_dict({
    "input_ids": tokenized_train["input_ids"],
    "attention_mask": tokenized_train["attention_mask"],
    "labels": tokenized_train["labels"]
})

val_dataset = Dataset.from_dict({
    "input_ids": tokenized_val["input_ids"],
    "attention_mask": tokenized_val["attention_mask"],
    "labels": tokenized_val["labels"]
})


In [74]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_total_limit=2,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-74-e99b9af9f943>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
# Assuming you have a list of all unique labels in your dataset
unique_labels = ['O', 'B-ENTITY', 'I-ENTITY']

# Create the label map (mapping labels to unique integer IDs)
label_map = {label: i for i, label in enumerate(unique_labels)}

# Verify the label map
print(label_map)


{'O': 0, 'B-ENTITY': 1, 'I-ENTITY': 2}
